In [674]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sklearn
import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.impute import SimpleImputer

In [675]:
# Settings
k_folds = 5

In [676]:
xls = pd.ExcelFile('Dataset - LBP RA.xlsx')
dataframe = pd.read_excel(xls, 'Training Dataset')
#dataframe = dataframe[(dataframe["Treatment"] == 1) | (dataframe["Treatment"] == 5)]
dataframe = dataframe[(dataframe["Treatment"] != 5)]
dataframe_original = dataframe.copy(True)
print(dataframe.info())

<class 'pandas.core.frame.DataFrame'>
Index: 897 entries, 0 to 1545
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Treatment                         897 non-null    int64  
 1   Fever                             879 non-null    float64
 2   Duration_of_pain                  877 non-null    float64
 3   Sick_leave                        897 non-null    int64  
 4   Earlier_hospitalization           897 non-null    int64  
 5   Workoverload                      255 non-null    float64
 6   Familiy_history                   897 non-null    int64  
 7   Depression                        897 non-null    int64  
 8   Extremely_nervous                 862 non-null    float64
 9   Stress                            897 non-null    int64  
 10  Relationship_with_colleagues      581 non-null    float64
 11  Irrational_thoughts_risk_lasting  857 non-null    float64
 12  Irrational_t

In [677]:
categorical_columns = ["Treatment", "Weightloss_per_year"]

boolean_columns = [
    "Fever",
    "Sick_leave",
    "Earlier_hospitalization",
    "Workoverload",
    "Familiy_history",
    "Depression",
    "Stress",
    "Uses_analgesics",
    "Uses_corticosteroids",
    "Serious_disease",
    "Neurogenic_signals",
    "Continuous_pain",
    "Nocturnal_pain",
    "Loss_muscle_strength",
    "Trauma",
    "Failure_symptoms",
    "Incoordination",
    "Paidwork",
]

ordinal_columns = [
    "Duration_of_pain",
    "Extremely_nervous",
    "Relationship_with_colleagues",
    "Irrational_thoughts_risk_lasting",
    "Irrational_thoughts_work",
    "Coping_strategy",
    "Kinesiophobia_physical_exercise",
    "Kinesiophobia_pain_stop",
    "Age",
    "neck_pain_intensity",
    "low_back_pain_intensity",
    "arm_left_pain_intensity",
    "arm_right_pain_intensity",
    "leg_left_pain_intensity",
    "leg_right_pain_intensity",
    "working_ability",
]

value_columns = ["Decreased_mobility"]

In [678]:
# Mapping integer colum
dataframe[value_columns] = dataframe[value_columns].astype("Int64")

# Mapping categories and boolean columns
dataframe[categorical_columns] = dataframe[categorical_columns].astype("category")
dataframe[boolean_columns] = dataframe[boolean_columns].astype("boolean")

# Mapping ordinal columns 
age_mapping = {
    "0-19": 0,
    "20-29": 1,
    "30-39": 2,
    "40-49": 3,
    "50-59": 4,
    "60-69": 5,
    "70-79": 6,
    ">=80": 7,
}

dataframe["Age"] = dataframe["Age"].replace(age_mapping)

for column in ordinal_columns:
    dataframe[[column]] = dataframe[[column]].astype("Int64")
    dataframe[column].fillna(-1, inplace=True)
    dataframe[column] = pd.Categorical(dataframe[column], categories=sorted(dataframe[column].unique()), ordered=True)

In [679]:
missing_percentages = dataframe_original.isnull().mean()
columns_to_remove = missing_percentages[missing_percentages > 0.7].index.tolist()
dataframe = dataframe.drop(columns=columns_to_remove)

categorical_columns = [
    col for col in categorical_columns if col not in columns_to_remove
]
ordinal_columns = [col for col in ordinal_columns if col not in columns_to_remove]
boolean_columns = [col for col in boolean_columns if col not in columns_to_remove]
value_columns = [col for col in value_columns if col not in columns_to_remove]

X = dataframe.drop(columns=["Treatment"])
imputer = SimpleImputer(strategy="most_frequent")
X_imputed = imputer.fit_transform(X)
y = dataframe["Treatment"]

minority_data = dataframe[(dataframe["Treatment"] != 1) & (dataframe["Treatment"] != 5)]
minority_data = pd.concat([minority_data] * 3)
minority_X = minority_data.drop(columns=["Treatment"])
minority_y = minority_data["Treatment"]
train_X = imputer.fit_transform(pd.concat([minority_X, X], axis=0))
train_y = pd.concat([minority_y, y], axis=0)

models = [
    ("Decision Tree", DecisionTreeClassifier().fit(train_X, train_y)),
    ("Random Forest", OneVsRestClassifier(RandomForestClassifier()).fit(train_X, train_y)),
    ("Gradient Boosting", OneVsRestClassifier(HistGradientBoostingClassifier()).fit(train_X, train_y)),
    ("KNN", KNeighborsClassifier().fit(train_X, train_y)),
    ("Logistic Regression", LogisticRegression(max_iter=4000, solver='saga').fit(train_X, train_y)), 
    ("Neural Network", MLPClassifier(max_iter=4000).fit(train_X, train_y)),
    ("SVM",SVC().fit(train_X, train_y)) 
]

model_scores = {}

for name, model in models:
    scores = cross_val_score(model, X_imputed, y, cv=k_folds, scoring="accuracy")
    model_scores[name] = scores
    print(f"{name}: Accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

average_accuracy = np.mean([scores.mean() for scores in model_scores.values()])
print(f"Average Accuracy Across All Models: {average_accuracy:.4f}")

ensemble_model = VotingClassifier(estimators=models, voting="hard")
scores = cross_val_score(ensemble_model, X_imputed, y, scoring="accuracy")

print(f"Ensemble Model (Voting): Accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

Random Forest: Accuracy: 0.6402 (+/- 0.3847)
Gradient Boosting: Accuracy: 0.6146 (+/- 0.3827)
KNN: Accuracy: 0.7001 (+/- 0.0286)
Logistic Regression: Accuracy: 0.6135 (+/- 0.3886)
Neural Network: Accuracy: 0.5722 (+/- 0.3053)
SVM: Accuracy: 0.7347 (+/- 0.0046)
Average Accuracy Across All Models: 0.6459
Ensemble Model (Voting): Accuracy: 0.6747 (+/- 0.2304)


In [680]:
# X_encoded = pd.get_dummies(dataframe[categorical_columns], drop_first=True)
# X = pd.concat(
#     [dataframe[value_columns + ordinal_columns + boolean_columns], X_encoded], axis=1
# )
# X_clean = X.dropna()

# y = dataframe["Treatment"]
# y_clean = y[X.index.isin(X_clean.index)]

# minority_data = dataframe[(dataframe["Treatment"] != 1) & (dataframe["Treatment"] != 5)]
# minority_data = pd.concat([minority_data] * 3)
# minority_X_encoded = pd.get_dummies(minority_data[categorical_columns], drop_first=True)
# minority_X = pd.concat(
#     [
#         minority_data[value_columns + ordinal_columns + boolean_columns],
#         minority_X_encoded,
#     ],
#     axis=1,
# )
# minority_X_clean = minority_X.dropna()

# minority_y = minority_data["Treatment"]
# minority_y_clean = minority_y[minority_X.index.isin(minority_X_clean.index)]


# X_Train = pd.concat([X_clean, minority_X_clean], axis=0)
# y_Train = pd.concat([y_clean, minority_y_clean], axis=0)

# # print(y_clean.info())
# # print(y_Train.head())

In [681]:
# # Decision Tree Model
# param_grid = {
#     "max_depth": [1, 2, 3, 4, 5, 10],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 4],
# }
# grid_search = GridSearchCV(
#     DecisionTreeClassifier(), param_grid, cv=cv, scoring="accuracy"
# )
# grid_search.fit(X_Train, y_Train)
# best_params = grid_search.best_params_
# # print("Best Score:", grid_search.best_score_)

# best_max_depth = grid_search.best_params_["max_depth"]
# best_min_samples_split = grid_search.best_params_["min_samples_split"]
# best_min_samples_leaf = grid_search.best_params_["min_samples_leaf"]

# tree_model = DecisionTreeClassifier(
#     max_depth=best_max_depth,
#     min_samples_split=best_min_samples_split,
#     min_samples_leaf=best_min_samples_leaf,
# )

# tree_model.fit(X_Train, y_Train)
# tree_predicted = cross_val_predict(tree_model, X, y, cv=cv)

# # Evaluation
# print("Simple k=" + str(cv) + " K fold CV")
# print("Decision Tree Model:")
# print(classification_report(y, tree_predicted))

In [682]:
# class_labels = [str(label) for label in tree_model.classes_]
# plt.figure(figsize=(135, 90))
# plot_tree(tree_model,max_depth=5, feature_names=X.columns, class_names=class_labels, filled=True, rounded=True)
# plt.show()

# Note that colors are based on the majority class in a leaf (with intensity being an indicator for how large this majority is over the others).

In [683]:
# # Histogram-based Gradient Boosting Classification Tree Model
# boosting_model = HistGradientBoostingClassifier(max_depth=5)
# boosting_model.fit(X_Train, y_Train)
# boosting_predicted = cross_val_predict(boosting_model, X, y, cv=cv)

# # Evaluation
# print("Simple k=" + str(cv) + " K fold CV")
# print("Histogram-based Gradient Boosting Classification Tree Model:")
# print(classification_report(y, boosting_predicted))

In [684]:
# # Random forest model
# rf_model = RandomForestClassifier(n_estimators=100, max_depth=10)
# rf_model.fit(X_Train, y_Train)
# rf_predicted = cross_val_predict(rf_model, X_clean, y_clean, cv=cv)

# # Evaluation
# print("Simple k=" + str(cv) + " K fold CV")
# print("Random forest model:")
# print(classification_report(y_clean, rf_predicted))

In [685]:
# # KNN model
# knn_model = KNeighborsClassifier(n_neighbors=7)  
# knn_model.fit(X_Train, y_Train)
# knn_predicted = cross_val_predict(knn_model, X_clean, y_clean, cv=cv)

# # Evaluation
# print("Simple k=" + str(cv) + " K fold CV")
# print("K-Nearest Neighbors model:")
# print(classification_report(y_clean, knn_predicted))

In [686]:
# # Ensemble model - Duplication
# ensemble_model = VotingClassifier(estimators=[
#     ('decision_tree', tree_model),
#     ('gradient_boosting', boosting_model),
#     ('random_forest', rf_model)
#     ,('knn',knn_model)
# ], voting='hard')

# ensemble_predicted = cross_val_predict(ensemble_model, X_clean, y_clean, cv=cv)

# print("Simple k=" + str(cv) + " K fold CV")
# print("Ensemble Model:")
# print(classification_report(y_clean, ensemble_predicted))

In [687]:
# # Ensemble model - No duplication
# # ------------------------------------------------------------------------
# # Decision Tree Model
# param_grid = {
#     "max_depth": [1, 2, 3, 4, 5, 10],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 4],
# }
# grid_search = GridSearchCV(
#     DecisionTreeClassifier(), param_grid, cv=cv, scoring="accuracy"
# )
# grid_search.fit(X, y)
# best_params = grid_search.best_params_
# # print("Best Score:", grid_search.best_score_)

# best_max_depth = grid_search.best_params_["max_depth"]
# best_min_samples_split = grid_search.best_params_["min_samples_split"]
# best_min_samples_leaf = grid_search.best_params_["min_samples_leaf"]

# tree_model = DecisionTreeClassifier(
#     max_depth=best_max_depth,
#     min_samples_split=best_min_samples_split,
#     min_samples_leaf=best_min_samples_leaf,
# )

# tree_model.fit(X_clean, y_clean)
# # ------------------------------------------------------------------------
# # Random forest model
# rf_model = RandomForestClassifier(max_depth=5)
# rf_model.fit(X_clean, y_clean)
# # ------------------------------------------------------------------------
# # Histogram-based Gradient Boosting Classification Tree Model
# boosting_model = HistGradientBoostingClassifier(max_depth=5)
# boosting_model.fit(X_clean, y_clean)
# # ------------------------------------------------------------------------

# ensemble_model = VotingClassifier(estimators=[
#     ('decision_tree', tree_model),
#     ('gradient_boosting', boosting_model),
#     ('random_forest', rf_model)
# ], voting='hard')

# ensemble_predicted = cross_val_predict(ensemble_model, X_clean, y_clean, cv=cv)

# print("Simple k=" + str(cv) + " K fold CV")
# print("Ensemble Model:")
# print(classification_report(y_clean, ensemble_predicted))